# Manejo de tablas con `dplyr`  
Este Notebook muestra alguna de las funciones más comunes del paquete dplyr, explicando su sintaxis y mostando algunos ejemplos de usos.

In [ ]:
#Instalar paquetes la primera vez:
#install.packages("tidyverse")
install.packages("dslabs")

In [ ]:
library(tidyverse)
library(dslabs)

## Tidyverse

Tidyverse agrupa un conjunto de paquetes que contienen funciones para el tratamiento y visualizacion de datos en R:  
- dplyr = manejo de tablas de datos 
- ggplot2 = creación de gráficos de alta calidad y personalizables
- readr = lectura de archivos con tablas de datos
- tibble = creación y manejo de tablas tipo *tibble* (una version mejorada de los data frames básicos)
- stringr = operaciones con *strings* (texto)
- tidyr = manupulación de tablas para hacelas _alargadas_ (agrupar las variables, pocas columnas y muchas observaciones), o _ensanchadas_ (proceso contrario)
- forcats = operaciones para manejo de variables categóricas
- purrr = aplicar operaciones a cada elemento de una columna o fila (entre otras cosas)

Tidyverse está muy extendido como el principal entorno para manejo de tablas de datos. Todos los paquetes comparten una sintaxis y un funcionamiento similares que permiten realizar las tareas con un codigo mas limpio, entendible y sin incompatibilidades.  

En este notebook se mostarán principalmente las funcionalidades de la librería **dplyr**, aunque complementando con alguna otra (p.ej. tidyr).  

Un resumen de todas las opciones y funciones que ofrece dplyr puede verse en [**esta cheat sheet**](https://raw.githubusercontent.com/rstudio/cheatsheets/main/data-transformation.pdf)

## dslabs / gapminder

dslabs es un paquete que cuenta con diversos datasets a los que se puede llamar directamente usando la librería. Estos datasets están especialmente pensados para ser usados en pruebas y tutoriales, con tablas de datos ya organizados de diferentes temáticas y estructuras  

En este caso los ejemplos usarán la tabla gapminder: un juego de palabras con el aviso que suena por megafonia en el metro de londres ("mind the gap"), y al mismo tiempo sobre las desigualdades ("gap") que hay entre paises en diferentes partes del mundo, y cómo han ido cambiando con el tiempo.

El impulsor de estos datos es Hans Rosling, que en [esta maravillosa charla TED](https://www.youtube.com/watch?v=hVimVzgtD6w) cambió la forma no solo de ver las diferencias entre primer/tercer mundo, si no tambien de lo que todo lo pueden contar los datos y lo importante de tenerlos accesibles.

In [ ]:
df <- gapminder
head(df)
tail(df)

## El pipe ( %>% )

Uno de los elementos más distintivos de tidyverse es el pipe (escrito como %>% ) 

Normalmente la forma de realizar varias operaciones seguidas en R es ejecutar una por línea (sobreescribiendo el objeto o creando varios), o bien anidando funciones a base de meter una funcion dentro de otra que a su vez está dentro de otra (con el riesgo de errores y confusión)  

Para ello, con tidyverse suele emplearse el pipe, que permite encadenar las funciones a ejecutar en fila, de forma que el resultado saliente de una entre directamente en la siguiente, consiguiendo un código más limpio y legible.

In [ ]:
#Calcular la media de una columna y redondear
#Metodo clásico
round(mean(df$life_expectancy))

In [ ]:
#Usando el pipe
df$life_expectancy %>% mean() %>% round()

Y por supuesto, tambien se puede guardar como objeto (da igual al principio o al final)

In [ ]:
mean_life_expectancy <- df$life_expectancy %>% mean() %>% round()

df$life_expectancy %>% mean() %>% round() -> mean_life_expectancy_alt 

Al comparar si son iguales:

In [ ]:
mean_life_expectancy == mean_life_expectancy_alt

## Operaciones básicas

**Nota:** cuando estamos trabajando con funciones de tidyverse sobre un dataframe, se puede llamar a las columnas directamente, sin necesidad de usar '$'

### Select (seleccionar columnas)

Sintaxis básica:  
**select**(el `dataframe` (entra con el pipe), `nombre de columna/s`  con las que quedarse)

In [ ]:
#viene bien recordar los nombres de las columnas
names(df)

In [ ]:
#Seleccionar solo algunas columnas
df %>% select(country, continent, region) %>% head()

In [ ]:
#Seleccionar por exclusión de otras (todas menos "continent" y "region")
df %>% select(-continent, -region) %>% head()

### Filter (filtrar filas)

Una función muy util que permite quedarte solo con las filas (casos) que cumplen un determinado criterio, similar al proceso de hacer un query en otros lenguajes

**Sintaxis básica:**

filter(el data frame, nombre de columna == 'criterio que cumplen las filas que te interesan')

El criterio puede ser cualquier comparador o que devuelva TRUE/FALSE, por ejemplo:  
- **==** que sea igual a un valor o palabra
- **!=** que sea diferente a un valor o palabra
- **>** **<** que sea mayor o menor que un valor
- **>=** **=<** que sea igual o mayor/menor a un valor
- **is.na()** que sea NA 
- **!is.na()** que NO sea NA 
- **%in%** que aparezca en una lista de elementos

Además se pueden encadenar argumentos:  
- **&**: condición 1 **y** condicion 2
- **|**: condición 1 **o** condicion 2

In [ ]:
#Selección sencilla
df %>% filter(country == 'France') %>% head()

In [ ]:
#Selección por dos criterios
df %>% filter(country == 'Spain' & year >= 2008)

In [ ]:
#Seleccionar que los NO tengan NAs en un campo (y ver solo los primeros resultados)
df %>% filter(!is.na(gdp)) %>% head()

###Sampling (muestreo aleatorio)
Las funciones de sampleo permiten seleccionar aleatoriamente registros de una tabla, bien sea por cantidad con `sample_n()` o por proporción con `sample_frac()`.  

**Sintaxis básica:**  
sample_X(`dataframe`, `cantidad/proporción`, `replace = TRUE/FALSE`)  

El parametro `replace` indica si un mismo registro se puede seleccionar mas de una vez (replace = True) o si se debe samplear sin repetición (replace = False)

In [ ]:
# Seleccionar 10 obsevaciones al azar

set.seed(123) # Aqui se fija una "semilla" para generar numeros random. Si la semilla es la misma, el resultado también
df %>% sample_n(10, replace = FALSE)

### Arrange (ordenar por una columna)

Arrange ordena todo el dataframe en función de los valores (numéricos, alfabéticos o de orden de factor) de una columna seleccionada.  

**Sintaxis básica:**  
arrange(el `dataframe`, `columna de ordenación`) 

nota: Por defecto, la ordenación es ascendente (de menor a mayor). Para una ordenación descendente, se incluiría el nombre de la columna dentro de la función `desc()`, de la siguiente manera:  
arrange(el `dataframe`, desc(`columna de ordenación`))

In [ ]:
#Ver los 10 años de España con mayor esperanza de vida
#Filtrar por pais = España, ordenar por esperanza de vida descendente, ver 10 primeros registros
df %>% filter(country == 'Spain') %>% arrange(desc(life_expectancy)) %>% head()

### Mutate (crear columnas)

Mutate permite crear nuevas columnas aplicando un criterio (normalmente transformaciones de una o varias columnas ya existentes)

**Sintaxis básica:**  
mutate(el `dataframe`, `Nueva_columna` = `Operacion para generarla`)

In [ ]:
#Calcular el PIB per capita a partir de dividir la columna gdp (gross domestic product) entre la de population
#Nota: para eliminar  primero las filas que tengan ALGUN valor NA (en cualquier columna), se usa na.omit()
df %>% mutate(gdp_per_capapita = gdp/population) %>% na.omit() %>%  head()

### Case_when (generar valores según múltiples condiciones)

La función case_when es muy util cuando se quiere, por ejemplo, crear una nueva variable que sea el resultado de aplicar varios argumentos condicionales. Por ejemplo, si queremos que nos añada un valor o una etiqueta dependiendo de si se cumple una u otra condición.  

**Sintaxis**

La sintaxis de case_when puede parecer inicialmente más complicada, pero una vez entendida resulta sencilla de usar y ampliar los casos.

de forma basica, sería:
case_when(`condición` ~ `resultado`)

En la condición se incluye un conjunto de datos (p.ej. una columna de una tabla), y el resultado de la función será una lista de datos de la misma longitud, los cuales mostrarán el `resultado` en los casos en los que se cumpla la `condición` impuesta.

Ejemplo: con los datos de 2011, ver qué paises serían considerados como desarrollados ( _developed_ ) o en vías de desarrollo ( _developing_ ) en base a si el cálculo de PIB per capita es > 12000 (developed) o menor (developing)

In [ ]:
# Crear tabla filtrada con datos solo de 2011
df_2011 <- df %>% filter(year == 2011)

Al aplicar la función case_when, el resultado es una lista de la misma longitud que el input

In [ ]:
case_when((df_2011$gdp / df_2011$population) >= 12000 ~ "DEVELOPED",
         (df_2011$gdp / df_2011$population) <= 12000 ~ "Developing")

Este resultado tiene 2 problemas:  
1) Una lista como tal no es útil, es mejor si se asigna a una nueva variable de la tabla.  
2) Hay casos que se escapan a las condiciones descritas (los NAs, por ejemplo).

Para ello, vamos a guardar el resultado como una nueva variable  **STATUS** en la tabla, pero esta vez definiendo como developing countries a los que tengan un resultado > 5000, y indicando dos nuevas condiciones.  
Primero, que los resultados que sean NA los marque como "No info".  
Segundo que los casos que no caigan dentro de ninguna de las condiciones se les asigne la etiqueta "other" (en este caso serían las economías del tercer mundo). Indicamos esos "casos restantes sin contemplar" utilizando como último argumento _TRUE_


**Nota:** las condiciones en case_when se aplican en el orden escrito, es decir, si un caso cumple la primera, ya no será considerado en las siguientes.

In [ ]:
df_2011$STATUS <- case_when((df_2011$gdp / df_2011$population) >= 12000 ~ "DEVELOPED",
                       (df_2011$gdp / df_2011$population) >= 5000 ~ "Developing",
                        is.na(df_2011$gdp / df_2011$population) ~ "No information",
                        TRUE ~ "Other")

head(df_2011)
table(df_2011$STATUS) #para ver un conteo de cuantos casos hay de cada tipo

### Un poco de repaso...
Teniendo las funciones principales, vamos a concatenar unas cuantas.

Ejemplo: paises europeos con mayor PIB per capita en el año 2011  
Para ello, en una sola linea continua, vamos a...
- seleccionar solo nombre de pais, continente, PIB (GDP) y población
- filtrar solo los que correspondan al año y la region
- calcular la nueva columna con el valor de PIB per capita
- sacar un ranking solo con las 10 con mayor valor de esa estadística (funcion top_n(), mezcla de sample_n() y arrange() )

In [ ]:
df2 <- df %>% select(c(country, year, population, gdp, continent)) %>% #seleccion de columnas
             filter(continent == 'Europe' & year == 2011) %>% #filtrar casos
             mutate(gdp_percap = gdp/population) %>% #crear nueva columna
             top_n(10, gdp_percap) #top 10 ordenado por gdp_percap
df2

## Group by (agrupar)

group_by permite agrupar las observaciones en funcion de una variable de interés, y calcular estadísticas en cada grupo (media, numero de casos, etc...)

Para ello, el procedimiento cuenta de 2 pasos: 

1) generar una tabla agrupada con la funcion `group_by()`, que en apariencia será igual que la original (pero ya tiene el criterio de agrupamiento como una propiedad "oculta") 

2) extraer un resumen o tabla agrupada con la funcion `summarise()`, donde se aplica una **función de agregación** que nos devolverá una variable "resumen" con la información que queramos calculada para cada grupo

In [ ]:
#Agrupar los datos por cada año
#resumir la tabla al promedio de esperanza de vida en gada grupo (año)
df %>% group_by(year) %>% 
    summarise("Avg life expectancy" = mean(life_expectancy))

In [ ]:
#Seleccionar solo los paises europeos
#agrupar los datos de todo el periodo por cada pais
#y crear una tabla resumiendo el promedio de PIB en cada pais para todos los años (sin contar NAs)

df %>% filter(continent == 'Europe') %>%
    group_by(country) %>%
    summarise('Mean GDP' = mean(gdp, na.rm = TRUE)) %>% 
    head(10)

#### Summarise_each

### Joins (uniones de tablas)

Las funciones de joins se utilizan para combinar datos de dos tablas distintas en una sola tabla, siempre y cuando ambas tablas compartan una columna con información equivalente que permita emparejar casos. Los joins son especialmente utiles en lenguajes de manejo de bases de datos (como SQL) para recopilar información que está repartida entre varias tablas distintas con algún sistema de vinculación.  

Los 4 tipos de joins más comunes son:  

- **Inner join:** la tabla resultante contiene información solo de los casos que aparezcan en ambas tablas.  
- **Full join:** se combinan todos los registros de las dos tablas. Los registros de una tabla que no aparezcan en la otra tendrán NAs en las variables correspondientes a esa otra tabla.  
- **Left/Right join:** a los registros de una de las tablas se le añade la información que tengan asociada en la otra tabla, o NAs si no aparecen.  

![types of joins](https://www.educative.io/cdn-cgi/image/f=auto,fit=contain,w=600/api/page/4815761290297344/image/download/5643241383264256)

Como segundo dataset, vamos a utilizar una tabla con datos de índice de felicidad ([World Happines Index](https://en.wikipedia.org/wiki/World_Happiness_Report)) para periodo 2010-2012 obtenidos de [este enlace](https://photius.com/rankings/happiness_country_rankings_2012.html).

Nota: para abrir la tabla se puede usar la función `read.csv2()` (porque el separador es ";") de R base, o bien como en este caso usar del paquete `readr` (tambien en Tidyverse) la función `read_delim()` indicando el delimitador 

In [ ]:
happiness_df <- read_delim("https://raw.githubusercontent.com/AngelArcones/R_for_friends/main/Tutoriales/Datasets/happiness_index.csv",
                 delim = ";", col_types = cols())
head(happiness_df, 10)

**Pequeño bonus track**  
Para comprobar si los paises de esta nueva tabla se puede usar la siguiente combinación (todo con comandos de R base).  
Primero, saber cuantos paises tiene la tabla `happiness_df`.  
Segundo, sacar con `%in%` una lista de booleanos (TRUE / FALSE) según si cada país de esta tabla aparece en la original, y obtener la suma del total de la lista.

In [ ]:
length(happiness_df$country)

In [ ]:
sum(happiness_df$country %in% df$country)

Esto funciona porque por defecto, para R (y otros lenguajes), TRUE = 1 y FALSE = 0

In [ ]:
TRUE + TRUE

In [ ]:
FALSE + FALSE

In [ ]:
1 + TRUE

In [ ]:
sum(c(TRUE, TRUE, FALSE, TRUE, FALSE))

Para averiguar cual es el pais que no coincide, usaré un filtro por lista de TRUE/FALSE:  
1) partiendo de la lista de valores de columna que quiero comprobar (happiness_df\$country)  
2) creo una lista de TRUE o FALSE según los valores que cumplen una condición: los paises de rd que NO aparecen en df (el símbolo `!` implica negación o contrario)  
3) al estar dentro de la selección de valores `[...]` de la lista inicial (rd\$country), el resultado son solo los elementros que coinciden con un TRUE

In [ ]:
happiness_df$country[!happiness_df$country %in% df$country]

Estos son los paises cuyos nombres en la tabla de happiness index no coinciden con la de datos mundiales. Para los ejemplos de uniones podemos continuar sin corregirlo, aunque para un caso real habría que realizar un emparejamiento mas detallado de equivalencias de nombres.

### Union interna (inner join) 
En primer lugar, vamos a probar a hacer una unión del los casos compartidos entre ambas tablas. En este caso nos referiremos a las tablas así:  
- df (datos globales) será la tabla de partida y por lo tanto la de la "izquierda"  
- rd será la segunda tabla, la de la "derecha"

**Sintaxis:** 
inner_join(`dataframe izquierda`, `dataframe derecha`, `by` = campo/s en común para unir)  

Por defecto, si no se especifica el `by`, buscará una columna con el mismo nombre en ambas tablas. Es preferible especificarselo, y es fundamental en casos donde el nombre no coincida. En ese caso, se haría por ejemplo `by = c("country" = "pais")`

In [ ]:
#union por "country"
inner_join(df, happiness_df, by = "country") %>% head(10)

El ejemplo anterior funciona, pero le une la misma información a todas las observaciones de cada país, que corresponde solo a un periodo concreto.  

Para solucionarlo, vamos a añadir a la tabla happiness_df un campo de año (2011). De esta manera, ahora se puede realizar la unión por más de un campo (país y año), y en ese caso irá haciéndolo jerárquicamente en el orden en el que se indiquen (primero queremos unión por país, y dentro de eso unión al año concreto

In [ ]:
#Unión usando las dos tablas originales
#Los campos de unión se indican con un vector c(...)
happiness_df$year <- 2011
inner_join(df, happiness_df, by = c("country", "year")) %>% head(10)

### Union de múltiples tablas  
Para mostrar un ejemplo más complejo, vamos a cargar dos tablas más:  
- Ranking de Human Developement Index para el mismo periodo, obtenido de [aquí](https://www.nationsonline.org/oneworld/human_development_2011.htm)  
- Lista de los paises de la [OECD](https://en.wikipedia.org/wiki/OECD)

In [ ]:
hdi_df <- read_delim("https://raw.githubusercontent.com/AngelArcones/R_for_friends/main/Tutoriales/Datasets/HDI_index.csv",
                 delim = ";", col_types = cols())
head(hdi_df, 5)

In [ ]:
oecd_df <- read_delim("https://raw.githubusercontent.com/AngelArcones/R_for_friends/main/Tutoriales/Datasets/OECD_countries.csv",
                 delim = ";", col_types = cols())
oecd_df

Y con estos 4 datasets (datos globales, happiness index, HDI index y OECD) la idea va a ser obtener una tabla final con toda la información económica y de desarrollo, para los paises miembros de la OECD. Para ello, los pasos son:  
1) Unir la información global con la de happines, coincidiendo pais y año (inner join)
2) A esa tabla, incorporarle la información disponible de HDI index (left join)
3) El resultado, combinarlo con la de OECD para quedarnos solo con los paises ahí incluidos (inner join)  

Todo esto se puede hacer en una sola línea, concatenando joins. El resultado final se guardará en un nuevo objeto `total_df`.

**NOTA**: en la tablacde HDI, la columna se llama "Country" con mayuscula. En ese caso se debe indicar que esa es la equivalente a "country"

In [ ]:
total_df <- inner_join(df, happiness_df, by = c("country", "year")) %>%
            left_join(hdi_df, by = c("country" = "Country")) %>%
            inner_join(oecd_df, by = "country") %>%
            arrange(happiness_ranking)
total_df

#### Comparación R base vs dplyr joins  
Por lo general, las funciones de dplyr no solo simplifican la sintaxis, si no que ademas tienen mejor performance. Aqui se muestra una comparación entre el tiempo que se tarda en hacer un inner join con R base (función `merge`) y con dplyr (función `inner_join`).  
Para ello, se crean dos datasets de pega con 1 millon de filas y 6 columnas que incluyen diferentes tipos de datos (character, integer y float). Ambas comparten el campo _id_ pero con diferente orden en cada una.  


In [ ]:
nsize = 1000000
df_1 <- data.frame("id" = seq(1,nsize),
                  "var_1A" = rnorm(n = nsize, mean = 50),
                  "var_1B" = sample(x = LETTERS, size = nsize, replace = T),
                  "var_1C" = sample(x = c("Yes", "No"), size = nsize, replace = T),
                  "var_1D" = sample(x = c(0:100), size = nsize, replace = T),
                  "var_1E" = sample(x = c(1:5), size = nsize, replace = T))
head(df_1)
print(nrow(df_1))

In [ ]:
nsize = 1000000
df_2 <- data.frame("id" = sample(x = c(0:nsize), size = nsize, replace = F),
                  "var_2A" = rnorm(n = nsize, mean = 5, sd = 2),
                  "var_2B" = sample(x = letters, size = nsize, replace = T),
                  "var_2C" = sample(x = c("Male", "Female"), size = nsize, replace = T),
                  "var_2D" = sample(x = c(2010:2020), size = nsize, replace = T),
                  "var_2E" = sample(x = c(1:5), size = nsize, replace = T))
head(df_2)
print(nrow(df_2))

Comparamos el tiempo de cada operación. El valor a tener en consideración es el de **elapsed** que indica el tiempo transcurrido.

In [ ]:
# R Base
system.time(merge(df_1, df_2, by = "id"))

In [ ]:
# dplyr
system.time(inner_join(df_1, df_2, by = "id"))

Sin embargo, si lo que se busca es reducir tiempos y maximizar performance (para datasets de gran tamaño) la mejor opción es usar el formato [data.table](https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.html), que permite optimizar cálculos y operaciones aplicados a las tablas.

### Pivoting / Reshapes (cambiar la distribución de la tabla)

El paquete **tidyr**, también parte de Tidyverse, permite cambiar de "forma" las tablas ("pivoting"), pasando de lo que se llama el formato _long_ (p.ej. por cada país, cada dato anual en una fila) a formato _wide_ (p.ej. por cada país, cada dato anual es una columna).  

Las operaciones de reshapes suelen ser conceptualmente complejas de por sí, y requieren tener una idea clara de la transformación deseada.  

Pese a lo abstracto del proceso, las funciones de **tidyr** (`pivot_wider`/`pivot_longer`) proporcionan una sintaxis mas intuitiva que las antiguas funciones que ofrecía **Tidyverse** (`gather`/`spread`), y que las del paquete **reshape2** (`melt`/`cast`)

Para empezar, esto sería un ejemplo de tabla **long**: una tabla con los datos de Francia y España de PIB (GDP) por año

In [ ]:
test_long <- df %>% filter(country %in% c("Spain", "France") &  year >= 2008 & year <= 2011) %>% 
             select(country, year, gdp)
test_long

Esta tabla se podría pasar a un formato **wide**, donde tengamos una fila para cada país, y una columna para cada año con el dato de GDP.  

La sintaxis para ello es:  
pivot_wider(`tabla long`,`names_from` = variable que configura las columnas, `values_from` = variable con los valores de interés)  

En este caso:  
- `names_from` = el año (una columna por año)  
- `values_from` = GDP (valores que queremos tener)

In [ ]:
test_long %>% pivot_wider(names_from = year, values_from = gdp)

Para mostrar el paso contrario (pasar de wide a long) sin recurrir a deshacer el proceso con la misma tabla, voy a crear una tabla inventada con localidades de muestreo.

In [ ]:
test_wide <- data.frame("location" = c("Forest", "Shrubland", "Grassland", "Dehesa"),
                        "val_2019" = rnorm(n = 4, mean = 4, sd = 0.7),
                        "val_2020" = rnorm(n = 4, mean = 3, sd = 1.2),
                        "val_2021" = rnorm(n = 4, mean = 3.5, sd = 0.7),
                        "val_2022" = rnorm(n = 4, mean = 4, sd = 0.7))
test_wide

Para pasarlo a formato **long**, se usara la función `pivot_longer()`. En ella indicaremos que queremos 

In [ ]:
test_wide %>% pivot_longer(cols = c("val_2019","val_2020","val_2021","val_2022"), names_to = "year")

In [ ]:
#para hacerlo un poco mas limpio

my_cols <- grep("val_", names(test_wide), value=TRUE) #Seleccionar nombres de columnas por patrón

test_wide_l <- test_wide %>% pivot_longer(cols = all_of(my_cols), names_to = "year") %>% #pivot a long
                             mutate(year = as.numeric(gsub("val_", "", year))) #eliminar el prefijo del año y hacerlo numerico


test_wide_l

Este paso de formato wide a long puede resultar especialmente útil cuando se quiere preparar una tabla de forma que los datos se puedan usar sencillamente en un gráfico de `ggplot` 

In [ ]:
options(repr.plot.width=10, repr.plot.height=5) #tamaño del gráfico

ggplot(test_wide_l, aes(x = year, y = value, group = location, color = location))+
    geom_line(size = 2)+
    geom_point(size = 4.5)+
    scale_color_manual(values = c("Forest" = "forestgreen",
                                  "Shrubland" = "goldenrod2",
                                  "Grassland" = "cyan4",
                                  "Dehesa" = "firebrick3"))+
    theme_minimal()